In [ ]:
#packages for processing data
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import numpy as np
import nltk
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

from collections import Counter
#from sklearn.feature_extraction.text import CountVectorizer

#packages to process data
import pandas as pd
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora

#display results
from IPython.display import display, HTML
from pprint import pprint

 **Load data file**

In [3]:
filepath = "reviews_Hotel1.csv"

In [4]:
data = pd.read_csv(filepath, sep=",", error_bad_lines=False, encoding='latin-1')

In [5]:
data.head()

,Review Rating,Review_Title,Topic
0,5.0,I had a pleasure of staying in this hotel for ...,Facilities Comfort Food Staff Location Service
1,5.0,A very lovely first visit to this iconic hotel...,Food Price Service
2,4.0,3 of us stayed at the Rhodes Hotel for 4 night...,Facilities Cleanliness Staff Location
3,5.0,Form the moment we arrived until we left we ex...,Food Staff Service
4,1.0,Well I am no strange to London's 5star hotels ...,Facilities Cleanliness Food Price Staff Service


In [6]:
# Remove unneccessary words that could become topics
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("great", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("room", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("stay", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("hotel", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("time", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("negative", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("Negative", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("positve", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("good", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("bad", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("one", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("make", ""))
data['Review_Title'] = data['Review_Title'].map(lambda x: str(x).replace("London", ""))


**Preprocess**

In [7]:
#pos tagging
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [8]:
def preProcess(text):
    # should return a list of tokens
    
    # word tokenisation, including punctuation removal
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # lowercasing
    tokens = [t.lower() for t in tokens]

    # stopword removal- benefits are it removes rare words, though bad for bigram relations
    stop = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop]

    # remove numbers and empty space
    digits = '0123456789'
    text = [t for t in text if t not in digits]

    
    
    # lemmatisation
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(t, get_wordnet_pos(t)) for t in tokens]
    tokens = [t for t in tokens if t] # ensure no empty space
    
    return tokens

In [9]:
def create_bigrams(words):
    bigram = gensim.models.Phrases(words, min_count=2, threshold=30)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

**Prepare data to train**

In [10]:
data['processed'] = data['Review_Title'].apply(preProcess)

In [11]:
data.head()

,Review Rating,Review_Title,Topic,processed
0,5.0,I had a pleasure of ing in this for 7 nights ...,Facilities Comfort Food Staff Location Service,"[pleasure, ing, 7, night, recently, perfect, e..."
1,5.0,A very lovely first visit to this iconic bar!...,Food Price Service,"[lovely, first, visit, iconic, bar, wonderful,..."
2,4.0,"3 of us ed at the Rhodes Hotel for 4 nights, i...",Facilities Cleanliness Staff Location,"[3, u, ed, rhodes, hotel, 4, night, location, ..."
3,5.0,Form the moment we arrived until we left we ex...,Food Staff Service,"[form, moment, arrive, left, experienced, abso..."
4,1.0,Well I am no strange to 's 5star s and when a ...,Facilities Cleanliness Food Price Staff Service,"[well, strange, 5star, new, come, along, eager..."


In [12]:
#Convert reviews from table to list
x_raw = data['processed'].values.tolist()

#Apply bi-gram
bigram_mod = create_bigrams(x_raw)
x = [bigram_mod[line] for line in x_raw]

x_train = [' '.join(i) for i in x]

#Create dictionary for gensim

dictionary = corpora.Dictionary(x)
corpus = [dictionary.doc2bow(text) for text in x]

Define Model

In [ ]:
!unzip mallet-2.0.8.zip

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=9, id2word=dictionary)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#save model
import joblib

#joblib.dump(ldamallet, 'lda9__b.jl')
#ldamallet = joblib.load('lda9__b.jl')

In [ ]:
pprint(ldamallet.show_topics(formatted=False, num_words=3))

[(0,
  [('location', 0.19801107834187973),
   ('breakfast', 0.06430161122205816),
   ('great', 0.05545011000897592)]),
 (1,
  [('close', 0.046818005663067964),
   ('walk', 0.043881759977028194),
   ('station', 0.039981518924211396)]),
 (2,
  [('breakfast', 0.12365199633860277),
   ('bar', 0.04507139499684013),
   ('food', 0.038777525138370676)]),
 (3,
  [('check', 0.03446124706134314),
   ('book', 0.032817662297855286),
   ('reception', 0.024806046191314053)]),
 (4,
  [('bed', 0.14377882038802775),
   ('small', 0.08333530598138451),
   ('comfortable', 0.07056793326452737)]),
 (5,
  [('service', 0.029942810208367643),
   ('make', 0.01888808244438074),
   ('amaze', 0.01763572156114103)]),
 (6,
  [('staff', 0.2964399149469386),
   ('friendly', 0.1196609005077542),
   ('helpful', 0.10492780407799918)]),
 (7,
  [('night', 0.03400896606947342),
   ('price', 0.03157625244442508),
   ('2', 0.025558200467368626)]),
 (8,
  [('bath', 0.03469363003249137),
   ('shower', 0.034488298214145126),
   (

Assign topic for new document

In [ ]:
unseen_document = ['The', 'price', 'is', 'very', 'cheap']
bow_vector = dictionary.doc2bow(unseen_document)
for index, score in sorted(ldamallet[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, ldamallet.print_topic(index, 5)))

Create results table

In [ ]:
def format_topics_sentences(ldamodel=model, corpus=corpus, texts=data):
   
    sent_topics_df = pd.DataFrame()

   
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: 
                wp = ldamodel.show_topic(topic_num)
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4)]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution']

    
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
    

topic_keywords = format_topics_sentences(ldamodel=model, corpus=corpus, texts=x_train)



In [ ]:
result = topic_keywords.reset_index()
result.columns = ['Document_No', 'Topic_assigned', 'Topic_Percentage', 'Text']

In [ ]:
#Convert topic number into topic words
result['Topic_assigned'] = result['Topic_assigned'].map(lambda x: str(x).replace('0.0', 'Location, Food'))
result['Topic_assigned'] = result['Topic_assigned'].map(lambda x: str(x).replace('1.0', 'Location'))
result['Topic_assigned'] = result['Topic_assigned'].map(lambda x: str(x).replace('2.0', 'Food'))
result['Topic_assigned'] = result['Topic_assigned'].map(lambda x: str(x).replace('3.0', 'Staff'))
result['Topic_assigned'] = result['Topic_assigned'].map(lambda x: str(x).replace('4.0', 'Comfort, Cleanliness'))
result['Topic_assigned'] = result['Topic_assigned'].map(lambda x: str(x).replace('5.0', 'Service'))
result['Topic_assigned'] = result['Topic_assigned'].map(lambda x: str(x).replace('6.0', 'Staff, Location'))
result['Topic_assigned'] = result['Topic_assigned'].map(lambda x: str(x).replace('7.0', 'Price'))
result['Topic_assigned'] = result['Topic_assigned'].map(lambda x: str(x).replace('8.0', 'Facilities'))


result = result[['Text', 'Topic_assigned', 'Topic_Percentage']]
result['Ground_Truth'] = data['Topic'].values

In [ ]:
result.head()

,Text,Topic_assigned,Topic_Percentage,Ground_Truth
0,pleasure ing 7 night recently perfect every wa...,Service,0.2340,Facilities Comfort Food Staff Location Service
1,lovely first visit iconic bar wonderful servic...,Service,0.2886,Food Price Service
2,3 u ed rhodes_hotel 4 night location take padd...,Price,0.1645,Facilities Cleanliness Staff Location
3,form moment_arrive left experienced absolute_p...,Service,0.2637,Food Staff Service
4,well strange 5star new come along eager try he...,Staff,0.2831,Facilities Cleanliness Food Price Staff Service


In [ ]:
#result.to_csv('result.csv', index=False)

**Show topics infered**

In [ ]:
from IPython.display import display, HTML

display(HTML(result[:1000].to_html()))

,Text,Topic_assigned,Topic_Percentage,Ground_Truth
0,pleasure ing 7 night recently perfect every way communication ing prompt efficient check breeze go spectacular lobby modern glass chandelier take elevator give upgrade junior_suite spectacular walk closet size could put small bed serve u nicely seven day decor refine oh bath carrera marble floor heat throughout rain shower die location turn 5 minute walk trafalgar_square quiet right outside embankment_tube stop would walk theater area numerous restaurant many major site eye_westminster abbey within walk_distance buffet breakfast service every morning pricy rate include couple night glass_wine sit front fire place lobby use spa include rate almost every night windchill day sightsee steam_sauna jacuzzi would soften weary muscle ed many 5_star around world top would return heartbeat next,Service,0.2340,Facilities Comfort Food Staff Location Service
1,lovely first visit iconic bar wonderful service without_intrusive delicious cocktail generally round indulgent experience well worth visit life experience though sure feel flush come cheap,Service,0.2886,Food Price Service
2,3 u ed rhodes_hotel 4 night location take paddington express heathrow like location clost partk walk_distance location bath small compare american hotel clean enjoy free wifi owner staff friendly helpful taxi resturant recomendations would,Price,0.1645,Facilities Cleanliness Staff Location
3,form moment_arrive left experienced absolute_perfection service excellance every member savoy staff famous personalise service rich heritage savoy truly best world eat din woud highly_recommend kaspers restaurant cannot wait return,Service,0.2637,Food Staff Service
4,well strange 5star new come along eager try heard corinthia via friend work industry say bar really something incident total 3 4th june second 4 night get cash go exchange dollar fly u pretty soon come felt light count almost 1000 obviously shock contact reception start investigation well couple day later checkout matter resolve make pay bill full upon return met manager jean_louis say matter still investigation give card get touch nothing ever resolve boast best best world pay premium reason expect premium service epically fail dent confidence industry even get formal_apology lovely would say risk 500 night risk,Staff,0.2831,Facilities Cleanliness Food Price Staff Service
5,staff helpful book wrong mistake change without hassle staff greet glass_prosecco lovely comfortable bed comfortable would definitely,Staff,0.1579,Facilities Comfort Staff
6,already step taxi get special feel welcome friendly smile help hand yet lot attitude ever organises well front_desk almost everywhere complicate slow check already reserve difficult swipe_card get key_card quickly forgotten use bar restaurant everything smart comfy nicely design service excellent well,Staff,0.1690,Facilities Comfort Food Staff Service
7,incredibly cool place delicious interest cocktail_dandelion bar follow incredible meal sea_container restaurant staff friendly service exceptional rumpus room top floor bar perfect place night cap cool atmosphere could recommend mondrian,Service,0.2300,Facilities Food Staff Service
8,savoy savoy every come back real surprise top service astonish place awesome city work several try many different place level every come back ask chose another living like travel back gentleman gentleman heart world,Service,0.2439,Facilities Service
9,location next park close harvey_nic harrod mean shopping nearby also restaurant mandarin_oriental best group perhaps even best property need tlc get service nice upgrade large use feel like real mo like get old ba first_class v new date know well bar nicer redid ago drunk biz pople mixed crowd dinner_heston close due flew issue week baloud basement cozy elegant compare still nyc property fantastic bangkok property worthy mo brand still mistake call designer redo reception bar first floor area seat touch impression much well,Service,0.2075,Facil

Visualisation of results

In [ ]:
from gensim.models.ldamodel import LdaModel

def convertldaMalletToldaGen(mallet_model):
    model_gensim = LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha) 
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

model = convertldaMalletToldaGen(ldamallet)

In [ ]:
#!pip install pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, dictionary=model.id2word)

In [ ]:
pyLDAvis.display(vis)